In [15]:
import pandas as pd
import scipy.optimize as opt
from datetime import datetime
import openpyxl

In [16]:
def calculate_coupon_payment_amount(face_value, coupon_rate, coupon_frequency):
    return face_value * coupon_rate / coupon_frequency

calculate_coupon_payment_amount(15_000, 0.0753, 4)

282.375

In [17]:
def generate_payment_dates(start_date, maturity_date, coupon_frequency):
    dates = pd.date_range(start=start_date, end=maturity_date, freq=f"{12//coupon_frequency}ME", )
    dates = list(dates.map(lambda x: x.replace(day=15)))

    maturity_date = pd.Timestamp(maturity_date)
    if maturity_date not in dates:
        dates.append(maturity_date)

    return dates

generate_payment_dates("1987-05-15", "1988-11-15", 2)

[Timestamp('1987-05-15 00:00:00'),
 Timestamp('1987-11-15 00:00:00'),
 Timestamp('1988-05-15 00:00:00'),
 Timestamp('1988-11-15 00:00:00')]

In [18]:
def populate_cashflows(purchase_price, face_value, coupon_rate, coupon_frequency, first_coupon_amount, settlement_date, first_coupon_date, maturity_date):
    cashflows = list()

    payment_amount = calculate_coupon_payment_amount(face_value, coupon_rate, coupon_frequency)

    # Payments will be on the same date of the month as the first coupon date. Separated by a fixed number of months.
    payment_dates = generate_payment_dates(first_coupon_date, maturity_date, coupon_frequency)
    cashflow_dates = [pd.Timestamp(settlement_date)] + payment_dates

    for index, payment_date in enumerate(cashflow_dates):
        elapsed_days = (payment_date - cashflows[-1]["Date"]).days if index > 0 else 0
        if index == 0:
            cashflows.append({'Date': payment_date, 'Cash Flow': -purchase_price, 'Elapsed Days': elapsed_days})
        elif index == 1:
            cashflows.append({'Date': payment_date, 'Cash Flow': first_coupon_amount, 'Elapsed Days': elapsed_days})
        elif index == len(cashflow_dates) - 1:
            cashflows.append({'Date': payment_date, 'Cash Flow': face_value + payment_amount, 'Elapsed Days': elapsed_days})
            break
        else:
            cashflows.append({'Date': payment_date, 'Cash Flow': payment_amount, 'Elapsed Days': elapsed_days})

        # Add in end of tax year if needed
        if index < len(cashflow_dates) - 1:
            next_end_of_tax_year = pd.Timestamp(f"{payment_date.year}-03-31") if payment_date < pd.Timestamp(f"{payment_date.year}-03-31") else pd.Timestamp(f"{payment_date.year + 1}-03-31")
            if next_end_of_tax_year < cashflow_dates[index+1]:
                cashflows.append({
                    'Date': next_end_of_tax_year,
                    'Cash Flow': 0,
                    'Elapsed Days': (next_end_of_tax_year - payment_date).days
                })

    next_end_of_tax_year = pd.Timestamp(f"{payment_dates[-1].year}-03-31") if payment_dates[-1].month < 3 else pd.Timestamp(f"{payment_dates[-1].year + 1}-03-31")
    cashflows.append({
        'Date': next_end_of_tax_year,
        'Cash Flow': 0,
        'Elapsed Days': (next_end_of_tax_year - payment_dates[-1]).days
    })

    return pd.DataFrame(cashflows).fillna(0)

# cashflows = populate_cashflows(50_000, 50_000, 0.0673, 4, 841.35, '2023-09-19', '2023-12-21', '2026-09-21')
cashflows = populate_cashflows(1_012_500, 1_000_000, 0.14, 2, 70_000, '1987-03-12', '1987-05-15', '1988-11-15')
# cashflows = populate_cashflows(20_000, 20_000, 0.0756, 4, 378, "2023-09-19", "2023-12-21", "2026-09-21")
cashflows

,Date,Cash Flow,Elapsed Days
0,1987-03-12,-1012500.0,0
1,1987-03-31,0.0,19
2,1987-05-15,70000.0,45
3,1987-11-15,70000.0,184
4,1988-03-31,0.0,137
5,1988-05-15,70000.0,45
6,1988-11-15,1070000.0,184
7,1989-03-31,0.0,136


In [19]:
def calc_daily_rate(ytm, frequency):
    true_rate = (1 + ytm/frequency) ** frequency - 1
    daily_rate = (1 + true_rate) ** (1 / 365) - 1

    return daily_rate
def PV_of_cashflow(ytm, cashflows, frequency):
    daily_rate = calc_daily_rate(ytm, frequency)
    PV_factor = cashflows.apply(lambda x: (1 + daily_rate) ** (-(x['Date'] - cashflows.loc[0, 'Date']).days), axis=1)
    PV_of_cashflows = cashflows['Cash Flow'] * PV_factor
    return PV_of_cashflows.sum()

def calculate_ytm(cashflows, first_guess, frequency):

    ytm = opt.fsolve(PV_of_cashflow, first_guess, (cashflows, frequency), xtol=1e-12)

    return ytm[0]

ytm = calculate_ytm(cashflows, 0.14, 2)
ytm

np.float64(0.16213707840816954)

In [20]:
PV_of_cashflow(0.16213707840816954, cashflows, 2)

np.float64(-5.3551048040390015e-09)

In [21]:
def populate_interest_principle_columns(cashflows, daily_rate):
    new_columns = list()

    for index, row in cashflows.iterrows():
        if index == 0:
            new_columns.append({
                "CurrentInterest": 0,
                "CurrentPrincipal": cashflows.loc[index, 'Cash Flow'],
                "CumulativeInterest": 0,
                "ClosingPrincipal": cashflows.loc[index, 'Cash Flow']
            })
            continue
        current_interest = ((1+daily_rate)**cashflows.loc[index, 'Elapsed Days'] - 1) * (-new_columns[- 1]['ClosingPrincipal'])
        current_principal = cashflows.loc[index, 'Cash Flow'] - current_interest
        new_columns.append({
            "CurrentInterest": current_interest,
            "CurrentPrincipal": current_principal,
            "CumulativeInterest": new_columns[-1]['CumulativeInterest'] + current_interest,
            "ClosingPrincipal": new_columns[-1]['ClosingPrincipal'] + current_principal
        })

    return cashflows.join(pd.DataFrame(new_columns))

cashflows_with_interest_principal = populate_interest_principle_columns(cashflows, 0.0004272139910217465)

In [22]:
def interest_to_balance_data(df):
    for index, row in df.iterrows():
        if row['Cash Flow'] != 0:
            df.loc[index, 'InterestToBalance'] = 0
            continue
        
        df.loc[index, 'InterestToBalance'] = df.loc[index, 'CumulativeInterest'] - df.loc[0:(index-1), 'InterestToBalance'].sum()

    return df

interest_to_balance_data(cashflows_with_interest_principal)

,Date,Cash Flow,Elapsed Days,CurrentInterest,CurrentPrincipal,CumulativeInterest,ClosingPrincipal,InterestToBalance
0,1987-03-12,-1012500.0,0,0.000000,-1.012500e+06,0.000000,-1.012500e+06,0.000000
1,1987-03-31,0.0,19,8250.205418,-8.250205e+03,8250.205418,-1.020750e+06,8250.205418
2,1987-05-15,70000.0,45,19809.115030,5.019088e+04,28059.320448,-9.705593e+05,0.000000
3,1987-11-15,70000.0,184,79354.235799,-9.354236e+03,107413.556247,-9.799136e+05,0.000000
4,1988-03-31,0.0,137,59051.312677,-5.905131e+04,166464.868924,-1.038965e+06,158214.663506
5,1988-05-15,70000.0,45,20162.596580,4.983740e+04,186627.465504,-9.891275e+05,0.000000
6,1988-11-15,1070000.0,184,80872.392320,9.891276e+05,267499.857824,1.421764e-01,0.000000
7,1989-03-31,0.0,136,-0.008503,8.503428e-03,267499.849320,1.506798e-01,101034.980396


In [23]:
def generate_spreadsheet_page(path, bond_code, purchase_price, face_value, coupon_rate, coupon_frequency, first_coupon_amount, settlement_date, first_coupon_date, maturity_date):

    cashflows = populate_cashflows(purchase_price, face_value, coupon_rate, coupon_frequency, first_coupon_amount, settlement_date, first_coupon_date, maturity_date)

    ytm = calculate_ytm(cashflows, coupon_rate, coupon_frequency)

    daily_rate = calc_daily_rate(ytm, coupon_frequency)

    cashflows_with_interest_principal = populate_interest_principle_columns(cashflows, daily_rate)

    cashflows_with_interest_principal = interest_to_balance_data(cashflows_with_interest_principal)

    print(cashflows_with_interest_principal)
    writer = pd.ExcelWriter(path, engine="openpyxl", mode="w")
    cashflows_with_interest_principal.to_excel(writer, sheet_name=bond_code, startrow=5, index=False)

    writer.close()

In [ ]:
file_name = 'workbench/test_data.xlsx'

data = pd.read_excel(file_name)
print(data)
output_file_name = file_name.split('.')[0] + "_YTM_Calculation_" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".xlsx"


for index, row in data.iterrows():
    print(f"Doing {row['BondCode']}")

    generate_spreadsheet_page(output_file_name, row["BondCode"], row["PurchaseAmount"], row["FaceValue"], row["CouponRate"], row["CouponFrequency"], row["FirstCouponAmount"], row["SettlementDate"], row["FirstCouponDate"], row["MaturityDate"])



  BondCode  PurchaseAmount  FaceValue  CouponRate  CouponFrequency  \
0    PCTHB           20000      20000      0.0756                4   
1    PCTHC           15000      15000      0.0753                4   
2   WNZ2T2           50000      50000      0.0673                4   

   FirstCouponAmount          SettlementDate         FirstCouponDate  \
0             378.00 1970-01-01 12:00:45.188 1970-01-01 12:00:45.281   
1             282.25 1970-01-01 12:00:45.188 1970-01-01 12:00:45.281   
2             841.25 1970-01-01 12:00:45.148 1970-01-01 12:00:44.971   

             MaturityDate  
0 1970-01-01 12:00:46.286  
1 1970-01-01 12:00:46.651  
2 1970-01-01 12:00:48.989  
Doing PCTHB
                     Date  Cash Flow  Elapsed Days  CurrentInterest  \
0 1970-01-01 12:00:45.188   -20000.0             0          0.00000   
1 1970-01-01 12:00:46.286      378.0             0          0.00000   
2 1970-03-31 00:00:00.000        0.0            88        357.52764   

   CurrentPrincipal  

/tmp/ipykernel_614016/2899082913.py:14: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  ytm = opt.fsolve(PV_of_cashflow, first_guess, (cashflows, frequency), xtol=1e-12)
/tmp/ipykernel_614016/2899082913.py:14: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  ytm = opt.fsolve(PV_of_cashflow, first_guess, (cashflows, frequency), xtol=1e-12)
/tmp/ipykernel_614016/2899082913.py:14: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  ytm = opt.fsolve(PV_of_cashflow, first_guess, (cashflows, frequency), xtol=1e-12)
